In [1]:
import os
import re
import numpy as np
import pandas as pd
from datasets import load_dataset, Dataset, ClassLabel
from langchain.document_loaders import PyPDFLoader
from utils import read_json
import torch
from transformers import DistilBertTokenizer, DistilBertForSequenceClassification, TrainingArguments, Trainer
from sklearn.metrics import accuracy_score, f1_score
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
def get_whole_pdf(file_data):
    whole_pdf = ""
    for i in range(len(file_data)):
        whole_pdf += file_data[i].page_content

    return whole_pdf

In [3]:
configs = read_json('configs.json')
DATA_PATH = configs["DATA_PATH"]
DB_PATH = configs['DB_PATH']
files_list = [file for file in os.listdir(DATA_PATH) if os.path.isfile(os.path.join(DATA_PATH, file))]

def read_pdf(files_list, data_path=DATA_PATH):
    content_dict = {}

    for file_name in files_list:
        file_path = data_path + file_name
        loader = PyPDFLoader(file_path)
        content = loader.load()
        whole_pdf = ""
        for i in range(len(content)):
            whole_pdf += content[i].page_content

        whole_pdf = re.sub(r'[：。\n]', '', whole_pdf)
        content_dict[file_name] = whole_pdf

    return content_dict

In [4]:
files_list

['34944_高鐵_促參.pdf',
 '38005_核四_政策.pdf',
 '39243_核四_品質.pdf',
 '39477_核四_料件.pdf',
 '46365_高鐵_基金.pdf',
 '48052_大客車_逃生門.pdf',
 '48746_大客車_安全門.pdf',
 '49490_核四_延宕.pdf',
 '49627_核四_停建.pdf',
 '49676_大客車_駕照.pdf',
 '52922_大客車_超時.pdf',
 '54210_高鐵_出資.pdf',
 '54376_高鐵_航發.pdf',
 '54561_高鐵_機電.pdf',
 '58930_核四_封存.pdf']

In [5]:
pdf_contents = read_pdf(files_list)

In [6]:
def content_extraction(input, pattern):

    # Search using the pattern
    match = re.search(pattern, input, re.S)

    if match:
        result = match.group(1).strip()  # Use strip() to remove any leading/trailing whitespace
        print("Extraction succeed.")
        return result
    else:
        print("No match found")
        pass

In [7]:
pattern = r"貳、案   由(.*?)參、事實與理由"

extracted_content = {}
for filename, content in pdf_contents.items():
    extracted_content[filename] = content_extraction(content, pattern)

Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.
Extraction succeed.


In [8]:
def extract_label(filename):
    # Assuming the label is always after the first underscore and ends before the second underscore
    match = re.search(r'_(.*?)_', filename)
    if match:
        return match.group(1)  # This extracts text between the first pair of underscores
    return None  # In case the pattern does not match

In [9]:
formatted_data  = [{'label': extract_label(filename), 'text' : content } for filename, content in extracted_content.items()]
formatted_data 

[{'label': '高鐵',
  'text': '交通部於 87年間與臺灣高速鐵路公司簽訂「臺灣南北高速鐵路興建營運合約」及「臺灣南北高速鐵路站區開發合約 」，疏未預先審度該公司展延高鐵通車營運時程及遲延受領站區用地所生 損失之責任歸屬，並綢繆約定 相關處罰條款 ，嗣對政府權益保障恝置不察，一再同意該公司展延通車營運時程及受領站區用地， 致政府蒙受回饋金與租金收入減少及顧問費用增加等鉅額損失，均有違失，爰依監察法第 24條規定提案糾正'},
 {'label': '核四',
  'text': '核四封存後每年仍耗費數億元於資產維護管理，行政院及經濟部對外宣告核四重啟不可行，核四興建費用 2,833億元頇列為損失，行政院及 經濟部對核四政策之重大變動，導致資源嚴重浪費； 再者，經濟部宣布能源配比 (燃氣50%、燃煤30%、再生能源20%)之能源轉型政策，未經能源安全、能源經濟及環境影響等完整評估 ，復於再生能源發電量增加有限情況下， 以運轉中核電機組長期停機 方式減核 ，致近年火力發購電量逐年提高， 106年占比達84.4%，燃煤發電增幅 甚至高於燃氣，造成嚴重空氣污染；以及經濟部宣布 新能源政策 之前，並未評估其對電價之影響，迄 106年3月行政院始於國公營企業體檢小組會議評估 等情，均有違失，爰依法提案糾正'},
 {'label': '核四',
  'text': '台電公司 未落實「核四工程品質保證方案」，致龍門電廠試運轉時違規與注意改善事項層出不窮，如 抑壓池灌水作業 不當，致反應器廠房底層淹水 、壓力試驗合格之室內消防栓 系統，其 太平龍頭 竟脫落，致汽機廠房積水等 ，均嚴重衝擊國人對核能安全運轉之信心等情 ，確有諸多違失，爰依法提案糾正'},
 {'label': '核四',
  'text': '台灣電力股份有限公司 (下稱台電公司 )於本院調查核四廠一號機因施工測詴期間設備損壞而移用二號機相關設備之過程，所提供資料內容前後不一，設備組件損壞、採購及修復個數未能確實清查正確，顯見台電公司核四廠之料件管理系統紊亂，且回復本院公文一再發生資料正確性不足，核有怠失；另台電公司於 81年陳報核四興建計畫， 竟以69年所估算成本陳報， 未能如實報告核四建廠成本，致使政府無法確實評估該項投資計畫之成本效益，台電公司表示當時即考慮日後再以追加預算方式提出，此

In [10]:
label = []
for i in range(len(formatted_data)):
    current_label = formatted_data[i]['label']
    if current_label not in label:
        label.append(current_label)
    else:
        continue
label_feature = ClassLabel(names=label)
label_feature

ClassLabel(names=['高鐵', '核四', '大客車'], id=None)

In [11]:
dataset = Dataset.from_dict({
    'text': [item['text'] for item in formatted_data],
    'label': [item['label'] for item in formatted_data]
})
dataset

Dataset({
    features: ['text', 'label'],
    num_rows: 15
})

In [12]:
def preprocess_label(example):
    
    example['label'] = [label_feature.str2int(label) for label in example['label']]
    return example

In [13]:
dataset_processed = dataset.map(preprocess_label, batched=True)
dataset_split = dataset_processed.train_test_split(test_size=0.3)

Map:   0%|          | 0/15 [00:00<?, ? examples/s]

In [14]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
})

## distilbert-base-cased

In [15]:
model_ckpt = "distilbert-base-multilingual-cased"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# model = AutoModel.from_pretrained(model_ckpt).to(device)
tokenizer = DistilBertTokenizer.from_pretrained(model_ckpt)
model = DistilBertForSequenceClassification.from_pretrained(model_ckpt, num_labels=3)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-multilingual-cased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def extract_hidden_states(batch):
    # Place model inputs on the GPU
    inputs = {k:v.to(device) for k,v in batch.items() 
              if k in tokenizer.model_input_names}
    # Extract last hidden states
    with torch.no_grad():
        last_hidden_state = model(**inputs).last_hidden_state
    # Return vector for [CLS] token
    return {"hidden_state": last_hidden_state[:,0].cpu().numpy()}

In [17]:
def tokenize(batch):
    return tokenizer(batch["text"], padding=True, truncation=True)

In [18]:
dataset_split

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 5
    })
})

In [19]:
dataset_encoded = dataset_split.map(tokenize, batched=True, batch_size=None)

Map:   0%|          | 0/10 [00:00<?, ? examples/s]

Map:   0%|          | 0/5 [00:00<?, ? examples/s]

In [20]:
dataset_encoded.set_format("torch", 
                            columns=["input_ids",  "label","attention_mask"])

In [21]:
dataset_encoded

DatasetDict({
    train: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 10
    })
    test: Dataset({
        features: ['text', 'label', 'input_ids', 'attention_mask'],
        num_rows: 5
    })
})

In [22]:
def compute_metrics(pred):
    label = pred.label_ids
    preds = pred.predictions.argmax(-1)
    f1 = f1_score(label, preds, average="weighted")
    acc = accuracy_score(label, preds)
    return {"accuracy": acc, "f1": f1}

In [23]:
batch_size = 1
logging_steps = len(dataset_encoded["train"]) // batch_size
model_name = f"{model_ckpt}-finetuned-emotion"
training_args = TrainingArguments(output_dir=model_name,
                                  num_train_epochs=20,
                                  learning_rate=2e-5,
                                  per_device_train_batch_size=batch_size,
                                  per_device_eval_batch_size=batch_size,
                                  weight_decay=0.01,
                                  evaluation_strategy="epoch",
                                  disable_tqdm=False,
                                  logging_steps=logging_steps,
                                  push_to_hub=False, 
                                  log_level="error")

trainer = Trainer(model=model, args=training_args, 
                  compute_metrics=compute_metrics,
                  train_dataset=dataset_encoded["train"],
                  eval_dataset=dataset_encoded["test"],
                  tokenizer=tokenizer)

# Now you can train the model
trainer.train()


  0%|          | 0/200 [00:00<?, ?it/s]

{'loss': 1.1654, 'grad_norm': 7.438181400299072, 'learning_rate': 1.9e-05, 'epoch': 1.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.1690927743911743, 'eval_accuracy': 0.2, 'eval_f1': 0.06666666666666667, 'eval_runtime': 1.3755, 'eval_samples_per_second': 3.635, 'eval_steps_per_second': 3.635, 'epoch': 1.0}
{'loss': 1.0836, 'grad_norm': 7.246498107910156, 'learning_rate': 1.8e-05, 'epoch': 2.0}


  0%|          | 0/5 [00:00<?, ?it/s]

...

  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.1507492065429688, 'eval_accuracy': 0.6, 'eval_f1': 0.5800000000000001, 'eval_runtime': 1.6068, 'eval_samples_per_second': 3.112, 'eval_steps_per_second': 3.112, 'epoch': 19.0}
{'loss': 0.0278, 'grad_norm': 0.38011404871940613, 'learning_rate': 0.0, 'epoch': 20.0}


  0%|          | 0/5 [00:00<?, ?it/s]

{'eval_loss': 1.1445811986923218, 'eval_accuracy': 0.6, 'eval_f1': 0.5800000000000001, 'eval_runtime': 1.6145, 'eval_samples_per_second': 3.097, 'eval_steps_per_second': 3.097, 'epoch': 20.0}
{'train_runtime': 290.7177, 'train_samples_per_second': 0.688, 'train_steps_per_second': 0.688, 'train_loss': 0.3816986757516861, 'epoch': 20.0}


TrainOutput(global_step=200, training_loss=0.3816986757516861, metrics={'train_runtime': 290.7177, 'train_samples_per_second': 0.688, 'train_steps_per_second': 0.688, 'total_flos': 17593640136000.0, 'train_loss': 0.3816986757516861, 'epoch': 20.0})

In [24]:
trainer.save_model(f"./models/{model_ckpt}_finetune")
tokenizer.save_pretrained(f"./models/{model_ckpt}_finetune")

('./models/distilbert-base-multilingual-cased_finetune\\tokenizer_config.json',
 './models/distilbert-base-multilingual-cased_finetune\\special_tokens_map.json',
 './models/distilbert-base-multilingual-cased_finetune\\vocab.txt',
 './models/distilbert-base-multilingual-cased_finetune\\added_tokens.json')

In [25]:
# Load the model and tokenizer for classification
model_name = f"./models/{model_ckpt}_finetune"
model = DistilBertForSequenceClassification.from_pretrained(model_name)
tokenizer = DistilBertTokenizer.from_pretrained(model_name)
model.eval()


DistilBertForSequenceClassification(
  (distilbert): DistilBertModel(
    (embeddings): Embeddings(
      (word_embeddings): Embedding(119547, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (transformer): Transformer(
      (layer): ModuleList(
        (0-5): 6 x TransformerBlock(
          (attention): MultiHeadSelfAttention(
            (dropout): Dropout(p=0.1, inplace=False)
            (q_lin): Linear(in_features=768, out_features=768, bias=True)
            (k_lin): Linear(in_features=768, out_features=768, bias=True)
            (v_lin): Linear(in_features=768, out_features=768, bias=True)
            (out_lin): Linear(in_features=768, out_features=768, bias=True)
          )
          (sa_layer_norm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
          (ffn): FFN(
            (dropout): Dropout(p=0.1, inplace=False)


In [26]:
def get_embeddings(texts, model, tokenizer):
    # Tokenize the input texts
    inputs = tokenizer(texts, return_tensors="pt", padding=True, truncation=True)
    # Pass inputs through the model to get hidden states
    with torch.no_grad():
        outputs = model(**inputs, output_hidden_states=True)
    # Get the embeddings from the last hidden state
    hidden_states = outputs.hidden_states[-1]
    # Average the hidden states to get a single vector representation for each text
    embeddings = hidden_states.mean(dim=1)
    return embeddings


In [27]:
# Extract texts from the dictionary
texts = list(extracted_content.values())

# Get embeddings for all documents
embeddings = get_embeddings(texts, model, tokenizer)

# Compute cosine similarity between all pairs of embeddings
cosine_similarities = cosine_similarity(embeddings)

# Convert cosine similarity matrix to a DataFrame
doc_keys = list(extracted_content.keys())
cosine_sim_df = pd.DataFrame(cosine_similarities, index=doc_keys, columns=doc_keys)


In [28]:
cosine_sim_df

,34944_高鐵_促參.pdf,38005_核四_政策.pdf,39243_核四_品質.pdf,39477_核四_料件.pdf,46365_高鐵_基金.pdf,48052_大客車_逃生門.pdf,48746_大客車_安全門.pdf,49490_核四_延宕.pdf,49627_核四_停建.pdf,49676_大客車_駕照.pdf,52922_大客車_超時.pdf,54210_高鐵_出資.pdf,54376_高鐵_航發.pdf,54561_高鐵_機電.pdf,58930_核四_封存.pdf
34944_高鐵_促參.pdf,1.000000,0.517399,0.764754,0.513572,0.947289,0.710240,0.707691,0.714360,0.594111,0.795023,0.691997,0.860549,0.943300,0.904984,0.451911
38005_核四_政策.pdf,0.517399,1.000000,0.730181,0.963848,0.505538,0.346367,0.433638,0.885766,0.949206,0.428091,0.386443,0.702908,0.603124,0.490059,0.957997
39243_核四_品質.pdf,0.764754,0.730181,1.000000,0.726725,0.754922,0.746738,0.772009,0.855353,0.785310,0.779685,0.732627,0.822315,0.780985,0.773491,0.687476
39477_核四_料件.pdf,0.513572,0.963848,0.726725,1.000000,0.507019,0.334385,0.424184,0.882690,0.966257,0.426560,0.381147,0.695291,0.607074,0.480731,0.975364
46365_高鐵_基金.pdf,0.947289,0.505538,0.754922,0.507019,1.000000,0.635451,0.610503,0.696351,0.579203,0.712315,0.594977,0.837687,0.940212,0.926536,0.445955
48052_大客車_逃生門.pdf,0.710240,0.346367,0.746738,0.334385,0.635451,1.000000,0.935160,0.542990,0.404688,0.933270,0.931198,0.661363,0.648206,0.675919,0.290680
48746_大客車_安全門.pdf,0.707691,0.433638,0.772009,0.424184,0.610503,0.935160,1.000000,0.613204,0.490465,0.949078,0.962274,0.711191,0.663527,0.631965,0.383144
49490_核四_延宕.pdf,0.714360,0.885766,0.855353,0.882690,0.696351,0.542990,0.613204,1.000000,0.938732,0.631674,0.574900,0.870926,0.779460,0.668646,0.843010
49627_核四_停建.pdf,0.594111,0.949206,0.785310,0.966257,0.579203,0.404688,0.490465,0.938732,1.000000,0.506968,0.452441,0.766455,0.681286,0.559858,0.947874
49676_大客車_駕照.pdf,0.795023,0.428091,0.779685,0.426560,0.712315,0.933270,0.949078,0.631674,0.506968,1.000000,0.953397,0.769007,0.755776,0.720522,0.385640


In [30]:
cosine_sim_df['58930_核四_封存.pdf'].sort_values(ascending=False)

58930_核四_封存.pdf      1.000000
39477_核四_料件.pdf      0.975364
38005_核四_政策.pdf      0.957997
49627_核四_停建.pdf      0.947874
49490_核四_延宕.pdf      0.843010
39243_核四_品質.pdf      0.687476
54210_高鐵_出資.pdf      0.639749
54376_高鐵_航發.pdf      0.545871
34944_高鐵_促參.pdf      0.451911
46365_高鐵_基金.pdf      0.445955
54561_高鐵_機電.pdf      0.423752
49676_大客車_駕照.pdf     0.385640
48746_大客車_安全門.pdf    0.383144
52922_大客車_超時.pdf     0.341194
48052_大客車_逃生門.pdf    0.290680
Name: 58930_核四_封存.pdf, dtype: float32

In [29]:
cosine_sim_df['52922_大客車_超時.pdf'].sort_values(ascending=False)

52922_大客車_超時.pdf     1.000000
48746_大客車_安全門.pdf    0.962274
49676_大客車_駕照.pdf     0.953397
48052_大客車_逃生門.pdf    0.931198
39243_核四_品質.pdf      0.732627
34944_高鐵_促參.pdf      0.691997
54210_高鐵_出資.pdf      0.681063
54376_高鐵_航發.pdf      0.640373
54561_高鐵_機電.pdf      0.621717
46365_高鐵_基金.pdf      0.594977
49490_核四_延宕.pdf      0.574900
49627_核四_停建.pdf      0.452441
38005_核四_政策.pdf      0.386443
39477_核四_料件.pdf      0.381147
58930_核四_封存.pdf      0.341194
Name: 52922_大客車_超時.pdf, dtype: float32

In [33]:
cosine_sim_df['54210_高鐵_出資.pdf'].sort_values(ascending=False)

54210_高鐵_出資.pdf      1.000000
54376_高鐵_航發.pdf      0.919676
49490_核四_延宕.pdf      0.870926
34944_高鐵_促參.pdf      0.860549
46365_高鐵_基金.pdf      0.837687
39243_核四_品質.pdf      0.822315
54561_高鐵_機電.pdf      0.782419
49676_大客車_駕照.pdf     0.769007
49627_核四_停建.pdf      0.766455
48746_大客車_安全門.pdf    0.711191
38005_核四_政策.pdf      0.702908
39477_核四_料件.pdf      0.695291
52922_大客車_超時.pdf     0.681063
48052_大客車_逃生門.pdf    0.661363
58930_核四_封存.pdf      0.639749
Name: 54210_高鐵_出資.pdf, dtype: float32

In [34]:
cosine_sim_df['54376_高鐵_航發.pdf'].sort_values(ascending=False)

54376_高鐵_航發.pdf      1.000000
34944_高鐵_促參.pdf      0.943300
46365_高鐵_基金.pdf      0.940212
54210_高鐵_出資.pdf      0.919676
54561_高鐵_機電.pdf      0.883022
39243_核四_品質.pdf      0.780985
49490_核四_延宕.pdf      0.779460
49676_大客車_駕照.pdf     0.755776
49627_核四_停建.pdf      0.681286
48746_大客車_安全門.pdf    0.663527
48052_大客車_逃生門.pdf    0.648206
52922_大客車_超時.pdf     0.640373
39477_核四_料件.pdf      0.607074
38005_核四_政策.pdf      0.603124
58930_核四_封存.pdf      0.545871
Name: 54376_高鐵_航發.pdf, dtype: float32